In [1]:
from utils import datasets as ds

artists, tracks, trim_tracks = ds.load_tracks_and_artists()
features = ds.load_features(trim_tracks)

In [2]:
data = ds.display_dataset(trim_tracks)

interactive(children=(IntSlider(value=0, description='electronic', max=999), IntSlider(value=0, description='e…

In [3]:
features = ds.display_features(trim_tracks, data.result, features)

interactive(children=(Checkbox(value=False, description='chroma'), Checkbox(value=False, description='rmse'), …

In [4]:
train_data, test_data = ds.preprocessing(features.result)
model = ds.build_model(train_data)
ds.train_model(model, train_data)

Epoch 1/100
50/50 [==============================] - 29s 4ms/step - loss: 1.8675 - accuracy: 0.2973
Epoch 2/100
50/50 [==============================] - 0s 4ms/step - loss: 1.5798 - accuracy: 0.4310
Epoch 3/100
50/50 [==============================] - 0s 4ms/step - loss: 1.5142 - accuracy: 0.4559
Epoch 4/100
50/50 [==============================] - 0s 4ms/step - loss: 1.4489 - accuracy: 0.4826
Epoch 5/100
50/50 [==============================] - 0s 4ms/step - loss: 1.4017 - accuracy: 0.4997
Epoch 6/100
50/50 [==============================] - 0s 4ms/step - loss: 1.3738 - accuracy: 0.5117
Epoch 7/100
50/50 [==============================] - 0s 4ms/step - loss: 1.3241 - accuracy: 0.5265
Epoch 8/100
50/50 [==============================] - 0s 4ms/step - loss: 1.2858 - accuracy: 0.5529
Epoch 9/100
50/50 [==============================] - 0s 4ms/step - loss: 1.2439 - accuracy: 0.5672
Epoch 10/100
50/50 [==============================] - 0s 4ms/step - loss: 1.2359 - accuracy: 0.5604
Epoch 11

In [6]:
ds.test_model(model, test_data)

Results:
50/50 - 1s - loss: 4.9239 - accuracy: 0.4381
